In [2]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import csv
import pandas as pd
import random
from random import choice
from random import sample

import metis # lulz


# Data load

In [3]:
# grap init
G_dblp = nx.Graph()


# dblp.tsv
with open('data/dblp/com-dblp/out.com-dblp.tsv', 'r') as file:
    for line in file:
        source, target = line.strip().split(' ')
        G_dblp.add_edge(int(source), int(target))

num_edges = G_dblp.number_of_edges()
num_nodes = G_dblp.number_of_nodes()


print("number of directed edges in dblp.tsv:", num_edges)
print("number of directed nodes in dblp.tsv:", num_nodes)

number of directed edges in dblp.tsv: 1049866
number of directed nodes in dblp.tsv: 317080


## Choosing random nodes for landmark

In [4]:
def generate_landmark_list(graph, num_landmarks=20):
    """
    Generate a list of random landmarks from the given graph.

    Parameters:
    - graph: NetworkX graph
    - num_landmarks: Number of landmarks to generate (default is 20)
    Returns:
    - landmark_list: List of random landmarks
    """
    landmark_list = []

    for i in range(num_landmarks):
        random_node = choice(list(graph.nodes()))
        if random_node not in landmark_list:
            landmark_list.append(random_node)

    return landmark_list

# Example usage:


landmark_list = generate_landmark_list(G_dblp, num_landmarks=100)
print("Landmark List:", landmark_list)


Landmark List: [136786, 257310, 160540, 199718, 164810, 27307, 114520, 260305, 54324, 242326, 157636, 179152, 188408, 144990, 102525, 148168, 23672, 14315, 286873, 193936, 152924, 288644, 75313, 309922, 302278, 270050, 192816, 205189, 212542, 43762, 82164, 273447, 71028, 71488, 255899, 224929, 255254, 45947, 148521, 123747, 151979, 71598, 265746, 135225, 137722, 233971, 234533, 126223, 72038, 114922, 215910, 49116, 107988, 287272, 277843, 260154, 222109, 205296, 177097, 126282, 294830, 221155, 215089, 230684, 53119, 213133, 138260, 74238, 246977, 316783, 45301, 40731, 249686, 25225, 272791, 164271, 273206, 177999, 157934, 115250, 20008, 246719, 280812, 99937, 177862, 162377, 172717, 151859, 109826, 140770, 31920, 13337, 29863, 299147, 25118, 78782, 200134, 295630, 40822, 26701]


## Calculating each node's degree

In [5]:
# WITH REGARD OR AREA REPULSION
# Top 20 degree nodes regardless of area of repulsion
N = 20

# Get the top nodes with the highest degree
top_nodes_degree = [node for node in sorted(G_dblp.nodes(), key=G_dblp.degree, reverse=True)][:N]

# Print the degree for each of the top nodes
for node in top_nodes_degree:
    node_degree = G_dblp.degree(node)
    print(f"The degree of node {node} is {node_degree}")



The degree of node 3336 is 343
The degree of node 3345 is 296
The degree of node 167 is 290
The degree of node 14690 is 269
The degree of node 13941 is 264
The degree of node 30095 is 244
The degree of node 13842 is 230
The degree of node 865 is 227
The degree of node 3298 is 225
The degree of node 13811 is 221
The degree of node 15326 is 219
The degree of node 3346 is 218
The degree of node 3326 is 218
The degree of node 1827 is 215
The degree of node 1833 is 215
The degree of node 13953 is 215
The degree of node 45 is 208
The degree of node 7227 is 207
The degree of node 2486 is 201
The degree of node 6319 is 201


In [19]:
def calculate_degree_centrality(graph, N=20):
    """
    Get the top nodes with the highest degree in the graph.

    Parameters:
    - graph: NetworkX graph
    - N: Number of top nodes to retrieve (default is 20)
    """
    selected_nodes = []
    top_nodes_degree = [node for node in sorted(graph.nodes(), key=graph.degree, reverse=True)]

    for node in top_nodes_degree:
        if not any(prev_node in graph.neighbors(node) for prev_node in selected_nodes):
            selected_nodes.append(node)
            if len(selected_nodes) == N:
                break

    # for node in selected_nodes:
    #     print(f'Node: {node}, Degree: {graph.degree(node)}')
    return selected_nodes

# Example usage:
calculate_degree_centrality(G_dblp)


[3336,
 14690,
 13941,
 30095,
 13842,
 865,
 13811,
 15326,
 1827,
 1833,
 45,
 27138,
 1448,
 9401,
 7007,
 9800,
 10278,
 3300,
 2006,
 66102]

## Calculate each's node closeness centrality (approximation)

In [7]:
def approximate_closeness_centrality(graph, num_seeds):
    # Select a sample of random seed nodes
    seed_nodes = random.sample(list(graph.nodes()), num_seeds)

    # Initialize the closeness centrality dictionary
    approx_closeness_centrality = {node: 0.0 for node in graph.nodes()}

    # Perform BFS computations from each seed node
    for seed_node in seed_nodes:
        distances = nx.single_source_shortest_path_length(graph, seed_node)
        for node, distance in distances.items():
            approx_closeness_centrality[node] += distance

    # Normalize the closeness centrality by the number of seed nodes
    for node in graph.nodes():
        approx_closeness_centrality[node] /= num_seeds

    return approx_closeness_centrality

In [8]:
def calculate_closeness_centrality(graph, N=20, num_seeds=20):
    """
    Calculate the approximate closeness centrality for a given graph.

    Parameters:
    - graph: NetworkX graph
    - N: Number of top nodes to select (default is 20)
    - num_seeds: Number of random seed nodes to use (default is 20)
    """
    # Calculate approximate closeness centrality for each node
    approx_closeness_centrality = approximate_closeness_centrality(graph, num_seeds)

    # Initialize the list of selected nodes
    selected_nodes = []

    # Get the nodes sorted by closeness centrality
    sorted_nodes_closeness = [node for node in sorted(approx_closeness_centrality, key=approx_closeness_centrality.get) if approx_closeness_centrality[node] > 0]

    for node in sorted_nodes_closeness:
        # Check if the node is not in the neighborhood of any previously selected nodes
        if not any(prev_node in graph.neighbors(node) for prev_node in selected_nodes):
            selected_nodes.append(node)
            if len(selected_nodes) == N:
                break

    # Print the approximate closeness centrality for each of the selected nodes
    for node in selected_nodes:
        centrality = approx_closeness_centrality[node]
        print(f"The approximate closeness centrality of node {node} is {centrality}")
        return nx.closeness_centrality(graph)

calculate_closeness_centrality(G_dblp, N=20, num_seeds=20)

The approximate closeness centrality of node 15326 is 4.55
The approximate closeness centrality of node 9401 is 4.75
The approximate closeness centrality of node 7438 is 4.8
The approximate closeness centrality of node 167 is 4.8
The approximate closeness centrality of node 1197 is 4.85
The approximate closeness centrality of node 1642 is 4.85
The approximate closeness centrality of node 5087 is 4.85
The approximate closeness centrality of node 1448 is 4.9
The approximate closeness centrality of node 2977 is 4.9
The approximate closeness centrality of node 6319 is 4.9
The approximate closeness centrality of node 35862 is 4.9
The approximate closeness centrality of node 2024 is 4.95
The approximate closeness centrality of node 864 is 4.95
The approximate closeness centrality of node 480 is 4.95
The approximate closeness centrality of node 8623 is 4.95
The approximate closeness centrality of node 21005 is 5.0
The approximate closeness centrality of node 912 is 5.0
The approximate closene

## Calculate each's node betweeness centrality (approximation)

In [9]:
def approximate_betweenness_centrality(graph, num_seeds=20):
    # Select a sample of random seed nodes
    seed_nodes = random.sample(list(graph.nodes()), num_seeds)

    # Initialize the betweenness centrality dictionary
    approx_betweenness_centrality = {node: 0.0 for node in graph.nodes()}

    # Perform BFS computations from each seed node and accumulate dependencies
    for seed_node in seed_nodes:
        paths = nx.single_source_shortest_path(graph, source=seed_node)
        dependencies = {node: 0 for node in graph.nodes()}

        for path in paths.values():
            for node in path[1:-1]:  # Exclude the source and target nodes
                dependencies[node] += 1

        # Accumulate betweenness centrality using dependencies
        for node in graph.nodes():
            if node != seed_node:
                approx_betweenness_centrality[node] += dependencies[node]

    # Normalize the betweenness centrality by the number of seed nodes
    for node in graph.nodes():
        approx_betweenness_centrality[node] /= num_seeds

    return approx_betweenness_centrality


In [10]:
def calculate_betweenness_centrality(graph, num_seeds=20, N=20):
    """
    Calculate the approximate betweenness centrality for a given graph.

    Parameters:
    - graph: NetworkX graph
    - num_seeds: Number of random seed nodes to use (default is 20)
    - N: Number of top nodes to select (default is 20)
    """
    # Calculate approximate betweenness centrality for each node
    approx_betweenness_centrality = approximate_betweenness_centrality(graph, num_seeds)

    # # Count the number of nodes with zero betweenness centrality
    # num_nodes_with_zero_centrality = sum(1 for centrality in approx_betweenness_centrality.values() if centrality == 0)

    # Initialize the list of selected nodes
    selected_nodes = []

    # Get the nodes sorted by betweenness centrality
    sorted_nodes_betweenness = [node for node in sorted(approx_betweenness_centrality, reverse=True, key=approx_betweenness_centrality.get) if approx_betweenness_centrality[node] > 0]

    for node in sorted_nodes_betweenness:
        # Check if the node is not in the neighborhood of any previously selected nodes
        if not any(prev_node in graph.neighbors(node) for prev_node in selected_nodes):
            selected_nodes.append(node)
            if len(selected_nodes) == N:
                break

    # Print the approximate betweenness centrality for each of the selected nodes
    for node in selected_nodes:
        centrality = approx_betweenness_centrality[node]
        print(f"The approximate betweenness centrality of node {node} is {centrality}")

    # # Print the count of nodes with zero betweenness centrality
    # print(f"Number of nodes with zero betweenness centrality: {num_nodes_with_zero_centrality}")

# Example usage:
calculate_betweenness_centrality(G_dblp, num_seeds=20, N=20)


The approximate betweenness centrality of node 3298 is 16488.5
The approximate betweenness centrality of node 974 is 16089.4
The approximate betweenness centrality of node 67033 is 15878.55
The approximate betweenness centrality of node 26687 is 15872.65
The approximate betweenness centrality of node 11331 is 15866.55
The approximate betweenness centrality of node 66815 is 15865.75
The approximate betweenness centrality of node 69119 is 15864.3
The approximate betweenness centrality of node 58862 is 15864.25
The approximate betweenness centrality of node 182183 is 15862.15
The approximate betweenness centrality of node 205338 is 15857.55
The approximate betweenness centrality of node 98991 is 15856.2
The approximate betweenness centrality of node 223854 is 15854.85
The approximate betweenness centrality of node 277767 is 15854.85
The approximate betweenness centrality of node 136697 is 15248.8
The approximate betweenness centrality of node 68381 is 14986.7
The approximate betweenness c

# Partitioning

In [15]:
def partition_graph(graph, num_partitions):
    """
    Partition a graph using METIS.

    Parameters:
    - graph: NetworkX graph
    - num_partitions: Number of partitions

    Returns:
    - partitions: List of sets representing the node partitions
    - edge_cut: Number of edges cut during partitioning
    """

    # Perform METIS graph partitioning
    (edge_cut, parts) = metis.part_graph(graph, nparts=num_partitions)

    # Create a list of sets, where each set contains the nodes in a partition
    partition_sets = [set() for _ in range(num_partitions)]
    for node, partition in enumerate(parts):
        partition_sets[partition].add(node)

    return partition_sets, edge_cut

In [27]:
partition_graph(G_dblp, num_partitions=10)
# Partition the graph
partitions, _ = partition_graph(G_dblp, num_partitions=10)
increment = 0
# Calculate centrality measures for each partition
for partition in partitions:
    # Create a subgraph for the current partition
    subgraph = G_dblp.subgraph(partition)

    # Calculate and print the centrality measures for the subgraph
    print("Partition " + str(increment))
    closeness_centrality = calculate_closeness_centrality(subgraph)
    lowest_centrality_node = min(closeness_centrality, key=closeness_centrality.get)
    print("Lowest closeness centrality node:", lowest_centrality_node)
    increment += 1


Partition 0
The approximate closeness centrality of node 265069 is 0.05
The approximate closeness centrality of node 268634 is 0.05
The approximate closeness centrality of node 6795 is 0.05
The approximate closeness centrality of node 269156 is 0.05
The approximate closeness centrality of node 12312 is 0.05
The approximate closeness centrality of node 14161 is 0.05
The approximate closeness centrality of node 279786 is 0.05
The approximate closeness centrality of node 153884 is 0.05
The approximate closeness centrality of node 37881 is 0.05
The approximate closeness centrality of node 310354 is 0.05
The approximate closeness centrality of node 311164 is 0.05
The approximate closeness centrality of node 51938 is 0.05
The approximate closeness centrality of node 314981 is 0.05
The approximate closeness centrality of node 201861 is 0.05
The approximate closeness centrality of node 80678 is 0.05
The approximate closeness centrality of node 225752 is 0.05
The approximate closeness centralit

AttributeError: 'NoneType' object has no attribute 'get'